In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
.appName('new_app_name') \
.master('local[*]') \
.getOrCreate()

In [2]:
player_file_path =r"C:\Users\Timothy\OneDrive\桌面\BU Fall '24\Project\player_stats\per_game_stats.csv"
adv_player_file_path =r"C:\Users\Timothy\OneDrive\桌面\BU Fall '24\Project\player_stats\advanced_stats.csv"

players_df = spark.read.csv(player_file_path, header=True, inferSchema=True)
adv_players_df = spark.read.csv(adv_player_file_path, header=True, inferSchema=True)

In [3]:
from pyspark.sql import functions as F
from pyspark.sql import Window


window = Window.partitionBy("Player Reference", "Season")

# Add a column that flags whether Tm is 'TOT'
players_df = players_df.withColumn("is_tot", F.when(F.col("Tm") == "TOT", 1).otherwise(0))


players_df = players_df.withColumn("total_count", F.count("*").over(window))


players_df = players_df.filter(
    (F.col("Tm") == "TOT") | (F.col("total_count") == 1)
)

# Drop the helper columns ('is_tot', 'total_count') after filtering
players_df = players_df.drop("is_tot", "total_count")
##END

# Add a column that flags whether Tm is 'TOT'
adv_players_df = adv_players_df.withColumn("is_tot", F.when(F.col("Tm") == "TOT", 1).otherwise(0))

# Count how many entries exist for each 'Player Reference' and 'Season'
adv_players_df = adv_players_df.withColumn("total_count", F.count("*").over(window))

# Filter the DataFrame:

adv_players_df = adv_players_df.filter(
    (F.col("Tm") == "TOT") | (F.col("total_count") == 1)
)
adv_players_df = adv_players_df.drop("is_tot", "total_count")


In [ ]:
from pyspark.sql.window import Window

#ACTIVE PLAYER 1/0

# Check if the player has a record in the '2023-24' season to determine if they are active
active_players = players_df.filter(F.col('Season') == '2023-2024').select('Player Reference').distinct()

# Add a new column 'Active_Player' which checks if the player has played in '2023-2024'
players_df = players_df.withColumn(
    "Active_Player",
    F.when(F.col('Player Reference').isin([row['Player Reference'] for row in active_players.collect()]), 1).otherwise(0)
)

In [5]:
from pyspark.sql import functions as F

# Group by 'Player Reference' and count the distinct seasons played
season_count = players_df.groupBy('Player Reference').agg(F.countDistinct('Season').alias('Total_Seasons_Played'))
players_df = players_df.join(season_count, on='Player Reference', how='left')

In [6]:

window_spec = Window.partitionBy("Player Reference").orderBy("Season")

# Add a new column 'Season_Rank' which will indicate 1st, 2nd, 3rd, etc.
players_df = players_df.withColumn("Season_Rank", F.row_number().over(window_spec))

In [7]:
from pyspark.sql import functions as F

# Our data would start form 1979-1980 season.
start_season = '1946-1947'
end_season = '1978-1979'

# Filter the DataFrame to remove rows between the defined seasons
players_df = players_df.filter(~((F.col('Season') >= start_season) & (F.col('Season') <= end_season)))
adv_players_df = adv_players_df.filter(~((F.col('Season') >= start_season) & (F.col('Season') <= end_season)))


In [8]:
adv_players_df.filter(adv_players_df['Player'] == 'Kareem Abdul-Jabbar*').show()

+--------------------+---+----+---+---+------+----+-----+-----+-----+----+----+----+----+----+----+----+----+---+---+----+-----+----+----+----+----+----------------+---------+------+
|              Player|Pos| Age| Tm|  G|    MP| PER|  TS%| 3PAr|  FTr|ORB%|DRB%|TRB%|AST%|STL%|BLK%|TOV%|USG%|OWS|DWS|  WS|WS/48|OBPM|DBPM| BPM|VORP|Player Reference|   Season|League|
+--------------------+---+----+---+---+------+----+-----+-----+-----+----+----+----+----+----+----+----+----+---+---+----+-----+----+----+----+----+----------------+---------+------+
|Kareem Abdul-Jabbar*|  C|36.0|LAL| 80|2622.0|21.3|0.608|0.001|0.318| 7.8|16.8|12.7|11.5| 1.0| 2.9|13.5|25.1|5.9|3.1| 8.9|0.163| 2.8| 0.2| 3.0| 3.3|       abdulka01|1983-1984|   NBA|
|Kareem Abdul-Jabbar*|  C|32.0|LAL| 82|3143.0|25.3|0.639|0.001|0.344| 7.2|22.2|15.4|16.5| 1.2| 4.6|15.7|24.1|9.5|5.3|14.8|0.227| 4.8| 2.4| 7.2| 7.3|       abdulka01|1979-1980|   NBA|
|Kareem Abdul-Jabbar*|  C|41.0|LAL| 74|1695.0|12.9|0.511|0.005| 0.25| 7.3|14.2|11.0| 

In [9]:
print(players_df.count(), adv_players_df.count())

19375 19375


In [10]:
# Join two data sets, the standard one and the advanced one.
select_df = players_df.select(
    'Player',"Age", "G", "MP" , "3P%","TRB", 'FG%','FT%',"AST", "STL", "BLK", "TOV", 
     "PTS", "Total_Seasons_Played", "Season_Rank",'Player Reference', 'Season','Active_Player'
)

# "PER", "TS%", "OWS", "DWS", "VORP",

select_adv_df = adv_players_df.select(
    'WS','WS/48','Player Reference', 'Season'
)

data = select_df.join(select_adv_df, on=['Player Reference', 'Season'], how='inner')

data.show(5)

+----------------+---------+------------------+----+---+----+-----+---+-----+-----+---+---+---+---+---+--------------------+-----------+-------------+---+-----+
|Player Reference|   Season|            Player| Age|  G|  MP|  3P%|TRB|  FG%|  FT%|AST|STL|BLK|TOV|PTS|Total_Seasons_Played|Season_Rank|Active_Player| WS|WS/48|
+----------------+---------+------------------+----+---+----+-----+---+-----+-----+---+---+---+---+---+--------------------+-----------+-------------+---+-----+
|       champju01|2021-2022| Justin Champagnie|20.0| 36| 7.8|0.357|2.0|0.463|  1.0|0.3|0.2|0.1|0.2|2.3|                   3|          1|            1|0.8|0.138|
|       champju01|2022-2023| Justin Champagnie|21.0|  5| 6.8|  0.2|1.6|0.333| null|0.8|0.2|0.0|0.0|2.2|                   3|          2|            1|0.0|0.009|
|       champju01|2023-2024| Justin Champagnie|22.0|  2| 5.0|0.333|0.0|0.714| null|0.0|0.5|0.0|0.5|5.5|                   3|          3|            1|0.0|0.097|
|       dejeabr01|2015-2016|Bryce 

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

#Select Veteran Players (Players that have been in the league for 3+ seasons.)

# Define the window partitioning by 'Player Reference' and ordering by 'Season'
window_spec = Window.partitionBy("Player Reference").orderBy("Season")

# Add previous 1st, 2nd, and 3rd year data for all the categories, but only if 'Season_Rank' is >= 4
veteran_data = data.withColumn("Prev_G_1", F.when(F.col("Season_Rank") >= 4, F.lag("G", 1).over(window_spec))) \
       .withColumn("Prev_MP_1", F.when(F.col("Season_Rank") >= 4, F.lag("MP", 1).over(window_spec))) \
       .withColumn("Prev_3P%_1", F.when(F.col("Season_Rank") >= 4, F.lag("3P%", 1).over(window_spec))) \
       .withColumn("Prev_TRB_1", F.when(F.col("Season_Rank") >= 4, F.lag("TRB", 1).over(window_spec))) \
       .withColumn("Prev_FG%_1", F.when(F.col("Season_Rank") >= 4, F.lag("FG%", 1).over(window_spec))) \
       .withColumn("Prev_FT%_1", F.when(F.col("Season_Rank") >= 4, F.lag("FT%", 1).over(window_spec))) \
       .withColumn("Prev_AST_1", F.when(F.col("Season_Rank") >= 4, F.lag("AST", 1).over(window_spec))) \
       .withColumn("Prev_STL_1", F.when(F.col("Season_Rank") >= 4, F.lag("STL", 1).over(window_spec))) \
       .withColumn("Prev_BLK_1", F.when(F.col("Season_Rank") >= 4, F.lag("BLK", 1).over(window_spec))) \
       .withColumn("Prev_TOV_1", F.when(F.col("Season_Rank") >= 4, F.lag("TOV", 1).over(window_spec))) \
       .withColumn("Prev_PTS_1", F.when(F.col("Season_Rank") >= 4, F.lag("PTS", 1).over(window_spec))) \
       .withColumn("Prev_WS_1", F.when(F.col("Season_Rank") >= 4, F.lag("WS", 1).over(window_spec))) \
       .withColumn("Prev_WS/48_1", F.when(F.col("Season_Rank") >= 4, F.lag("WS/48", 1).over(window_spec))) \
       .withColumn("Prev_G_2", F.when(F.col("Season_Rank") >= 4, F.lag("G", 2).over(window_spec))) \
       .withColumn("Prev_MP_2", F.when(F.col("Season_Rank") >= 4, F.lag("MP", 2).over(window_spec))) \
       .withColumn("Prev_3P%_2", F.when(F.col("Season_Rank") >= 4, F.lag("3P%", 2).over(window_spec))) \
       .withColumn("Prev_TRB_2", F.when(F.col("Season_Rank") >= 4, F.lag("TRB", 2).over(window_spec))) \
       .withColumn("Prev_FG%_2", F.when(F.col("Season_Rank") >= 4, F.lag("FG%", 2).over(window_spec))) \
       .withColumn("Prev_FT%_2", F.when(F.col("Season_Rank") >= 4, F.lag("FT%", 2).over(window_spec))) \
       .withColumn("Prev_AST_2", F.when(F.col("Season_Rank") >= 4, F.lag("AST", 2).over(window_spec))) \
       .withColumn("Prev_STL_2", F.when(F.col("Season_Rank") >= 4, F.lag("STL", 2).over(window_spec))) \
       .withColumn("Prev_BLK_2", F.when(F.col("Season_Rank") >= 4, F.lag("BLK", 2).over(window_spec))) \
       .withColumn("Prev_TOV_2", F.when(F.col("Season_Rank") >= 4, F.lag("TOV", 2).over(window_spec))) \
       .withColumn("Prev_PTS_2", F.when(F.col("Season_Rank") >= 4, F.lag("PTS", 2).over(window_spec))) \
       .withColumn("Prev_WS_2", F.when(F.col("Season_Rank") >= 4, F.lag("WS", 2).over(window_spec))) \
       .withColumn("Prev_WS/48_2", F.when(F.col("Season_Rank") >= 4, F.lag("WS/48", 2).over(window_spec))) \
       .withColumn("Prev_G_3", F.when(F.col("Season_Rank") >= 4, F.lag("G", 3).over(window_spec))) \
       .withColumn("Prev_MP_3", F.when(F.col("Season_Rank") >= 4, F.lag("MP", 3).over(window_spec))) \
       .withColumn("Prev_3P%_3", F.when(F.col("Season_Rank") >= 4, F.lag("3P%", 3).over(window_spec))) \
       .withColumn("Prev_TRB_3", F.when(F.col("Season_Rank") >= 4, F.lag("TRB", 3).over(window_spec))) \
       .withColumn("Prev_FG%_3", F.when(F.col("Season_Rank") >= 4, F.lag("FG%", 3).over(window_spec))) \
       .withColumn("Prev_FT%_3", F.when(F.col("Season_Rank") >= 4, F.lag("FT%", 3).over(window_spec))) \
       .withColumn("Prev_AST_3", F.when(F.col("Season_Rank") >= 4, F.lag("AST", 3).over(window_spec))) \
       .withColumn("Prev_STL_3", F.when(F.col("Season_Rank") >= 4, F.lag("STL", 3).over(window_spec))) \
       .withColumn("Prev_BLK_3", F.when(F.col("Season_Rank") >= 4, F.lag("BLK", 3).over(window_spec))) \
       .withColumn("Prev_TOV_3", F.when(F.col("Season_Rank") >= 4, F.lag("TOV", 3).over(window_spec))) \
       .withColumn("Prev_PTS_3", F.when(F.col("Season_Rank") >= 4, F.lag("PTS", 3).over(window_spec))) \
       .withColumn("Prev_WS_3", F.when(F.col("Season_Rank") >= 4, F.lag("WS", 3).over(window_spec))) \
       .withColumn("Prev_WS/48_3", F.when(F.col("Season_Rank") >= 4, F.lag("WS/48", 3).over(window_spec)))

veteran_data = veteran_data.filter(F.col("Season_Rank") >= 4)
veteran_data = veteran_data.filter(F.col("Prev_G_3").isNotNull())
veteran_data = veteran_data.fillna(0)

In [12]:
from pyspark.sql import functions as F
for col in ['G', 'MP', '3P%', 'TRB', 'FG%', 'FT%', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'WS', 'WS/48']:
    veteran_data = veteran_data.withColumn(f"{col}_Percent_Change_1", 
                                           F.when(F.col(f"Prev_{col}_2") != 0, 
                                                  ((F.col(f"Prev_{col}_1") - F.col(f"Prev_{col}_2")) / F.col(f"Prev_{col}_2")) * 100)
                                           .otherwise(0))


for col in ['G', 'MP', '3P%', 'TRB', 'FG%', 'FT%', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'WS', 'WS/48']:
    veteran_data = veteran_data.withColumn(f"{col}_Percent_Change_2", 
                                           F.when(F.col(f"Prev_{col}_3") != 0, 
                                                  ((F.col(f"Prev_{col}_2") - F.col(f"Prev_{col}_3")) / F.col(f"Prev_{col}_3")) * 100)
                                           .otherwise(0))



In [13]:
veteran_data.show(10)

+----------------+---------+--------------------+----+---+----+-----+---+-----+-----+---+---+---+---+----+--------------------+-----------+-------------+----+-----+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+-----------------

In [14]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
##BEST so FAR\
veteran_data = veteran_data.withColumnRenamed('3P%', '3P_Ptg')
veteran_data = veteran_data.filter((F.col('G') > 20) & (F.col('MP') > 10))   

# Define the feature columns (last 3 years)
feature_columns = [
    "Prev_G_1", "Prev_MP_1", "Prev_3P%_1", "Prev_TRB_1", "Prev_FG%_1", "Prev_FT%_1", "Prev_AST_1", "Prev_STL_1", "Prev_BLK_1", "Prev_TOV_1", "Prev_PTS_1", 
    "Prev_WS_1", "Prev_WS/48_1",
    # 新增變化百分比特徵  
    # 前年與三年前的變化
    "G_Percent_Change_2", "MP_Percent_Change_2", "3P%_Percent_Change_2", "TRB_Percent_Change_2", "FG%_Percent_Change_2", 
    "FT%_Percent_Change_2", "AST_Percent_Change_2", "STL_Percent_Change_2", "BLK_Percent_Change_2", "TOV_Percent_Change_2", "PTS_Percent_Change_2", 
    "WS_Percent_Change_2", "WS/48_Percent_Change_2",
    # 去年與前年的變化
    "G_Percent_Change_1", "MP_Percent_Change_1", "3P%_Percent_Change_1", "TRB_Percent_Change_1", "FG%_Percent_Change_1", 
    "FT%_Percent_Change_1", "AST_Percent_Change_1", "STL_Percent_Change_1", "BLK_Percent_Change_1", "TOV_Percent_Change_1", "PTS_Percent_Change_1", 
    "WS_Percent_Change_1", "WS/48_Percent_Change_1"
]


# Define the target columns (current year)
target_columns = ['G', 'MP', '3P_Ptg', 'TRB', 'FG%', 'FT%', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'WS', 'WS/48']

# Step 1: Combine features into a single feature vector using VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_transformed = assembler.transform(veteran_data)


# Split the data into train/test sets
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=42)
train_data.cache()

# Create a dictionary to store models and predictions
rf_models = {}
rf_predictions = {}


In [15]:
train_data.show()

+----------------+---------+--------------------+----+---+----+------+----+-----+-----+---+---+---+---+----+--------------------+-----------+-------------+----+-----+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+---------------

In [16]:
# Loop through each target column, train the Random Forest model, and make predictions
for target in target_columns:
    rf = RandomForestRegressor(featuresCol="features", labelCol=target, numTrees=100)
    
    # Fit the model
    rf_model = rf.fit(train_data)
    rf_models[target] = rf_model  # Store the trained model
    
    # Make predictions on test data
    rf_pred = rf_model.transform(test_data)
    rf_predictions[target] = rf_pred
    
    # Evaluate the model
    evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(rf_pred)
    print(f"Random Forest RMSE for {target}: {rmse}")
    
    r2_evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    r2 = r2_evaluator.evaluate(rf_pred)
    print(f"Random Forest R-squared for {target}: {r2}")

Random Forest RMSE for G: 14.959280016758695
Random Forest R-squared for G: 0.11059015460683863
Random Forest RMSE for MP: 5.069690869730306
Random Forest R-squared for MP: 0.6093645111845165
Random Forest RMSE for 3P_Ptg: 0.1117290192407262
Random Forest R-squared for 3P_Ptg: 0.5074670920654323
Random Forest RMSE for TRB: 1.2728309627082242
Random Forest R-squared for TRB: 0.7623003984216392
Random Forest RMSE for FG%: 0.039371362254641724
Random Forest R-squared for FG%: 0.5328508981792206
Random Forest RMSE for FT%: 0.07351425503332783
Random Forest R-squared for FT%: 0.5091245973410697
Random Forest RMSE for AST: 0.9599888620424696
Random Forest R-squared for AST: 0.7876704426308921
Random Forest RMSE for STL: 0.2706700673587422
Random Forest R-squared for STL: 0.6467201109489524
Random Forest RMSE for BLK: 0.27907974130362045
Random Forest R-squared for BLK: 0.7526910160971009
Random Forest RMSE for TOV: 0.44080456091888187
Random Forest R-squared for TOV: 0.6995847410915768
Rando

In [43]:
# Step 3: Train a linear regression model for each target variable
feature_columns = [
    "Prev_G_1", "Prev_MP_1", "Prev_3P%_1", "Prev_TRB_1", "Prev_FG%_1", "Prev_FT%_1", "Prev_AST_1", "Prev_STL_1", "Prev_BLK_1", "Prev_TOV_1", "Prev_PTS_1", 
    "Prev_WS_1", "Prev_WS/48_1",
    # 新增變化百分比特徵  
    # 前年與三年前的變化
    "G_Percent_Change_2", "MP_Percent_Change_2", "3P%_Percent_Change_2", "TRB_Percent_Change_2", "FG%_Percent_Change_2", 
    "FT%_Percent_Change_2", "AST_Percent_Change_2", "STL_Percent_Change_2", "BLK_Percent_Change_2", "TOV_Percent_Change_2", "PTS_Percent_Change_2", 
    "WS_Percent_Change_2", "WS/48_Percent_Change_2",
    # 去年與前年的變化
    "G_Percent_Change_1", "MP_Percent_Change_1", "3P%_Percent_Change_1", "TRB_Percent_Change_1", "FG%_Percent_Change_1", 
    "FT%_Percent_Change_1", "AST_Percent_Change_1", "STL_Percent_Change_1", "BLK_Percent_Change_1", "TOV_Percent_Change_1", "PTS_Percent_Change_1", 
    "WS_Percent_Change_1", "WS/48_Percent_Change_1"
]


# Define the target columns (current year)
target_columns = ['G', 'MP', '3P_Ptg', 'TRB', 'FG%', 'FT%', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'WS', 'WS/48']

# Step 1: Combine features into a single feature vector using VectorAssembler
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_transformed = assembler.transform(veteran_data)


# Split the data into train/test sets
train_data, test_data = df_transformed.randomSplit([0.7, 0.3], seed=42)

models = {}
predictions = {}
for target in target_columns:
    lr = LinearRegression(featuresCol="features", labelCol=target, maxIter=10)
    model = lr.fit(train_data)
    models[target] = model  # Store the trained model
    
    # Make predictions on test data
    pred = model.transform(test_data)
    predictions[target] = pred
    
    # Step 4: Evaluate the model
    evaluator = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(pred)
    print(f"Root Mean Squared Error (RMSE) for {target}: {rmse}")
    
    evaluator_r2 = RegressionEvaluator(labelCol=target, predictionCol="prediction", metricName="r2")
    r2 = evaluator_r2.evaluate(pred)
    print(f"R-squared for {target}: {r2}")



Root Mean Squared Error (RMSE) for G: 15.077873940999444
R-squared for G: 0.09643215910966296
Root Mean Squared Error (RMSE) for MP: 5.116303380734953
R-squared for MP: 0.6021482097749438
Root Mean Squared Error (RMSE) for 3P_Ptg: 0.1177108276973298
R-squared for 3P_Ptg: 0.4533163233493225
Root Mean Squared Error (RMSE) for TRB: 1.1861344718753415
R-squared for TRB: 0.7935785394983623
Root Mean Squared Error (RMSE) for FG%: 0.03800764185592585
R-squared for FG%: 0.5646520688411214
Root Mean Squared Error (RMSE) for FT%: 0.07176897150531619
R-squared for FT%: 0.5321554287129343
Root Mean Squared Error (RMSE) for AST: 0.8656729498409851
R-squared for AST: 0.8273423760691238
Root Mean Squared Error (RMSE) for STL: 0.25505439861638446
R-squared for STL: 0.6863075337501974
Root Mean Squared Error (RMSE) for BLK: 0.24469509662601388
R-squared for BLK: 0.809877374474805
Root Mean Squared Error (RMSE) for TOV: 0.4280528752462412
R-squared for TOV: 0.7167142909453414
Root Mean Squared Error (RM

In [17]:
predict_data = veteran_data.filter(veteran_data['Season'] == '2023-2024')

In [18]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Create the DataFrame (df) for demonstration
# Assuming df is the DataFrame you are working on

# Add 1 year to the 'Season' column and 1 to 'Age' column
predict_data = predict_data.withColumn("Season", F.concat(F.expr("substring(Season, 1, 4) + 1"), F.lit("-"), F.expr("substring(Season, 6, 9) + 1"))) \
       .withColumn("Age", F.col("Age") + 1)



# Move the previous percent changes to Percent_Change_2 columns
predict_data = predict_data.withColumn("G_Percent_Change_2", F.col("G_Percent_Change_1")) \
       .withColumn("MP_Percent_Change_2", F.col("MP_Percent_Change_1")) \
       .withColumn("3P%_Percent_Change_2", F.col("3P%_Percent_Change_1")) \
       .withColumn("TRB_Percent_Change_2", F.col("TRB_Percent_Change_1")) \
       .withColumn("FG%_Percent_Change_2", F.col("FG%_Percent_Change_1")) \
       .withColumn("FT%_Percent_Change_2", F.col("FT%_Percent_Change_1")) \
       .withColumn("AST_Percent_Change_2", F.col("AST_Percent_Change_1")) \
       .withColumn("STL_Percent_Change_2", F.col("STL_Percent_Change_1")) \
       .withColumn("BLK_Percent_Change_2", F.col("BLK_Percent_Change_1")) \
       .withColumn("TOV_Percent_Change_2", F.col("TOV_Percent_Change_1")) \
       .withColumn("PTS_Percent_Change_2", F.col("PTS_Percent_Change_1")) \
       .withColumn("WS_Percent_Change_2", F.col("WS_Percent_Change_1")) \
       .withColumn("WS/48_Percent_Change_2", F.col("WS/48_Percent_Change_1"))

# Calculate the percent change for each column and add to the new columns
predict_data = predict_data.withColumn("G_Percent_Change_1", ((F.col("G") - F.col("Prev_G_1")) / F.col("Prev_G_1")) * 100) \
       .withColumn("MP_Percent_Change_1", ((F.col("MP") - F.col("Prev_MP_1")) / F.col("Prev_MP_1")) * 100) \
       .withColumn("3P%_Percent_Change_1", ((F.col("3P_Ptg") - F.col("Prev_3P%_1")) / F.col("Prev_3P%_1")) * 100) \
       .withColumn("TRB_Percent_Change_1", ((F.col("TRB") - F.col("Prev_TRB_1")) / F.col("Prev_TRB_1")) * 100) \
       .withColumn("FG%_Percent_Change_1", ((F.col("FG%") - F.col("Prev_FG%_1")) / F.col("Prev_FG%_1")) * 100) \
       .withColumn("FT%_Percent_Change_1", ((F.col("FT%") - F.col("Prev_FT%_1")) / F.col("Prev_FT%_1")) * 100) \
       .withColumn("AST_Percent_Change_1", ((F.col("AST") - F.col("Prev_AST_1")) / F.col("Prev_AST_1")) * 100) \
       .withColumn("STL_Percent_Change_1", ((F.col("STL") - F.col("Prev_STL_1")) / F.col("Prev_STL_1")) * 100) \
       .withColumn("BLK_Percent_Change_1", ((F.col("BLK") - F.col("Prev_BLK_1")) / F.col("Prev_BLK_1")) * 100) \
       .withColumn("TOV_Percent_Change_1", ((F.col("TOV") - F.col("Prev_TOV_1")) / F.col("Prev_TOV_1")) * 100) \
       .withColumn("PTS_Percent_Change_1", ((F.col("PTS") - F.col("Prev_PTS_1")) / F.col("Prev_PTS_1")) * 100) \
       .withColumn("WS_Percent_Change_1", ((F.col("WS") - F.col("Prev_WS_1")) / F.col("Prev_WS_1")) * 100) \
       .withColumn("WS/48_Percent_Change_1", ((F.col("WS/48") - F.col("Prev_WS/48_1")) / F.col("Prev_WS/48_1")) * 100)

# Create new columns for the previous stats (Prev_ columns)
predict_data = predict_data.withColumn("Prev_G_1", F.col("G")) \
       .withColumn("Prev_MP_1", F.col("MP")) \
       .withColumn("Prev_3P%_1", F.col("3P_Ptg")) \
       .withColumn("Prev_TRB_1", F.col("TRB")) \
       .withColumn("Prev_FG%_1", F.col("FG%")) \
       .withColumn("Prev_FT%_1", F.col("FT%")) \
       .withColumn("Prev_AST_1", F.col("AST")) \
       .withColumn("Prev_STL_1", F.col("STL")) \
       .withColumn("Prev_BLK_1", F.col("BLK")) \
       .withColumn("Prev_TOV_1", F.col("TOV")) \
       .withColumn("Prev_PTS_1", F.col("PTS")) \
       .withColumn("Prev_WS_1", F.col("WS")) \
       .withColumn("Prev_WS/48_1", F.col("WS/48"))


predict_data = predict_data.withColumn("Season_Rank", F.col("Season_Rank") + 1) \
                .withColumn("Total_Seasons_Played", F.col("Total_Seasons_Played") + 1)

predict_data = predict_data.drop("G", "MP", "3P_Ptg", "TRB", "FG%", "FT%", "AST", "STL", "BLK", "TOV", "PTS", "WS", "WS/48")

In [19]:
#Result
predict_data.show(20)

+----------------+-------------+--------------------+----+--------------------+-----------+-------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+--------+---------+----------+----------+----------+----------+----------+----------+----------+----------+----------+---------+------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+----------------------+-------------------+-------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------

In [20]:
# Assuming new_data is the new dataset you want to predict on

# Dictionary to store predictions on the new data

feature_columns = [
    "Prev_G_1", "Prev_MP_1", "Prev_3P%_1", "Prev_TRB_1", "Prev_FG%_1", "Prev_FT%_1", "Prev_AST_1", "Prev_STL_1", "Prev_BLK_1", "Prev_TOV_1", "Prev_PTS_1", 
    "Prev_WS_1", "Prev_WS/48_1",
    # 新增變化百分比特徵  
    # 前年與三年前的變化
    "G_Percent_Change_2", "MP_Percent_Change_2", "3P%_Percent_Change_2", "TRB_Percent_Change_2", "FG%_Percent_Change_2", 
    "FT%_Percent_Change_2", "AST_Percent_Change_2", "STL_Percent_Change_2", "BLK_Percent_Change_2", "TOV_Percent_Change_2", "PTS_Percent_Change_2", 
    "WS_Percent_Change_2", "WS/48_Percent_Change_2",
    # 去年與前年的變化
    "G_Percent_Change_1", "MP_Percent_Change_1", "3P%_Percent_Change_1", "TRB_Percent_Change_1", "FG%_Percent_Change_1", 
    "FT%_Percent_Change_1", "AST_Percent_Change_1", "STL_Percent_Change_1", "BLK_Percent_Change_1", "TOV_Percent_Change_1", "PTS_Percent_Change_1", 
    "WS_Percent_Change_1", "WS/48_Percent_Change_1"
]


new_data_predictions = {}

target_columns = ['G', 'MP', '3P_Ptg', 'TRB', 'FG%', 'FT%', 'AST', 'STL', 'BLK', 'TOV', 'PTS', 'WS', 'WS/48']

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features", handleInvalid="skip")
df_transformed = assembler.transform(predict_data)

for target in target_columns:
    # Get the trained model for this target
    rf_model = rf_models[target]
    
    # Make predictions on the new data
    new_rf_pred = rf_model.transform(df_transformed)
    
    # Select relevant columns for storing predictions (including prediction column and some original columns)
    new_data_predictions[target] = new_rf_pred.select("prediction", "Player Reference")
    
    # Optionally: show the top 5 rows of predictions
    #print(f"Predictions for {target}:")
    #new_data_predictions[target].show(5)
    

In [25]:
final_df = predict_data.select("Player Reference", "Season", "Player","Age")
final_df = final_df.join(new_data_predictions['G'].select('Player Reference'), on='Player Reference', how='inner')
final_df.count()

236

In [26]:
# Show the final DataFrame with all predictions combined


for target in target_columns:
    # Get the predictions DataFrame for this target
    pred_df = new_data_predictions[target]
    
    # Rename the prediction column to reflect the target variable
    pred_col_name = f"predict_{target}"
    pred_df = pred_df.withColumnRenamed("prediction", pred_col_name)
    
    # Join the predictions to the final DataFrame
    final_df = final_df.join(pred_df.select("Player Reference", pred_col_name), on=['Player Reference'], how='left')


In [42]:
#Result

final_df.orderBy(col("predict_PTS").desc()).show()

+----------------+-------------+--------------------+----+-----------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|Player Reference|       Season|              Player| Age|        predict_G|        predict_MP|     predict_3P_Ptg|       predict_TRB|        predict_FG%|       predict_FT%|       predict_AST|       predict_STL|        predict_BLK|       predict_TOV|       predict_PTS|        predict_WS|      predict_WS/48|
+----------------+-------------+--------------------+----+-----------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+
|       jokicni01|2024.0-2025.0|        Nikola Jokić|29.0| 64.89268073812

In [28]:
mvp_2000_2010_path =r"C:\Users\Timothy\OneDrive\桌面\BU Fall '24\Project\player_stats\2001-2010 MVP Data.csv"
mvp_2010_2021_path =r"C:\Users\Timothy\OneDrive\桌面\BU Fall '24\Project\player_stats\2010-2021 MVP Data.csv"
mvp_2022_2023_path =r"C:\Users\Timothy\OneDrive\桌面\BU Fall '24\Project\player_stats\2022-2023 MVP Data.csv"

mvp_2000_2010 = spark.read.csv(mvp_2000_2010_path, header=True, inferSchema=True)
mvp_2010_2021 = spark.read.csv(mvp_2010_2021_path, header=True, inferSchema=True)
mvp_2022_2023 = spark.read.csv(mvp_2022_2023_path, header=True, inferSchema=True)

mvp_voting = mvp_2000_2010.union(mvp_2010_2021).union(mvp_2022_2023).distinct()


mvp_voting.show()

+---+----+--------------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+
|_c0|Rank|              Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS| TRB| AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|
+---+----+--------------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+
|  8|  9T|       Carlos Boozer| 25|UTA|  0.0|    7.0|   1290|0.005| 74|34.6|20.9|11.7| 3.0|0.9|0.3|0.561| null|0.685| 9.9|0.185|2007|
|  8|   9|        James Harden| 26|HOU|  0.0|    9.0|   1310|0.007| 82|38.1|29.0| 6.1| 7.5|1.7|0.6|0.439|0.359| 0.86|13.3|0.204|2016|
| 12|  13|    Andrei Kirilenko| 22|UTA|  0.0|    2.0|   1230|0.002| 78|37.1|16.5| 8.1| 3.1|1.9|2.8|0.443|0.338| 0.79|11.6|0.192|2004|
|  9|  9T|           Ray Allen| 29|SEA|  0.0|   41.0|   1270|0.032| 78|39.3|23.9| 4.4| 3.7|1.1|0.1|0.428|0.376|0.883|10.7|0.168|2005|
|  3|   4|       Tracy McGrady| 23|ORL|  4.0|  427.0|   1190|0

In [29]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import RandomForestRegressor

mvp_voting = mvp_voting.fillna(0)

#Pick '04 '08 '12 '16 '20 as test data years, others train.

train_years = [2000, 2001, 2002, 2003, 2005, 2006, 2007, 2009, 2010, 2011, 2013, 2014, 2015, 2017, 2018, 2019, 2021, 2022, 2023]
test_years = [2004, 2008, 2012, 2016, 2020]

# Select the features and the target column
feature_columns = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48']
target_column = 'Share'


assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data_assembled = assembler.transform(mvp_voting)

# Split the data into training and test sets
train_data = data_assembled.filter(data_assembled['year'].isin(train_years))
test_data = data_assembled.filter(data_assembled['year'].isin(test_years))


rf = RandomForestRegressor(featuresCol="features", labelCol=target_column, numTrees=150)


rf_model = rf.fit(train_data)

# Make predictions on the test data
predictions = rf_model.transform(test_data)



predictions.show(10)


Root Mean Squared Error (RMSE) on test data: 0.21303818660711618
R-squared on test data: 0.44763487116750555
+---+----+--------------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+-------------------+
|_c0|Rank|              Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS| TRB| AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|            features|         prediction|
+---+----+--------------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+-------------------+
|  8|   9|        James Harden| 26|HOU|  0.0|    9.0|   1310|0.007| 82|38.1|29.0| 6.1| 7.5|1.7|0.6|0.439|0.359| 0.86|13.3|0.204|2016|[82.0,38.1,29.0,6...|  0.382222642003529|
| 12|  13|    Andrei Kirilenko| 22|UTA|  0.0|    2.0|   1230|0.002| 78|37.1|16.5| 8.1| 3.1|1.9|2.8|0.443|0.338| 0.79|11.6|0.192|2004|[78.0,37.1,16.5,8...|0.07859564103774953|
|  8|   9|      

In [36]:
predictions.filter(predictions['year'] == 2016).orderBy(col("prediction").desc()).show()

+---+----+-----------------+---+---+-----+-------+-------+-----+---+----+----+---+----+---+---+-----+-----+-----+----+-----+----+--------------------+-------------------+
|_c0|Rank|           Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS|TRB| AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|            features|         prediction|
+---+----+-----------------+---+---+-----+-------+-------+-----+---+----+----+---+----+---+---+-----+-----+-----+----+-----+----+--------------------+-------------------+
|  0|   1|    Stephen Curry| 27|GSW|131.0| 1310.0|   1310|  1.0| 79|34.2|30.1|5.4| 6.7|2.1|0.2|0.504|0.454|0.908|17.9|0.318|2016|[79.0,34.2,30.1,5...| 0.6923687836766527|
|  4|   5|     Kevin Durant| 27|OKC|  0.0|  147.0|   1310|0.112| 72|35.8|28.2|8.2| 5.0|1.0|1.2|0.505|0.387|0.898|14.5| 0.27|2016|[72.0,35.8,28.2,8...| 0.4712994514121042|
|  8|   9|     James Harden| 26|HOU|  0.0|    9.0|   1310|0.007| 82|38.1|29.0|6.1| 7.5|1.7|0.6|0.439|0.359| 0.86|13.3|0.204|2016|[82.0,38.1,29.0,

In [37]:
predictions.filter(predictions['year'] == 2020).orderBy(col("prediction").desc()).show()

+---+----+--------------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+--------------------+
|_c0|Rank|              Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS| TRB| AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|            features|          prediction|
+---+----+--------------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+--------------------+
|  0|   1|Giannis Antetokou...| 25|MIL| 85.0|  962.0|   1010|0.952| 63|30.4|29.5|13.6| 5.6|1.0|1.0|0.553|0.304|0.633|11.1|0.279|2020|[63.0,30.4,29.5,1...|  0.4983905568875384|
|  2|   3|        James Harden| 30|HOU|  0.0|  367.0|   1010|0.363| 68|36.5|34.3| 6.6| 7.5|1.8|0.9|0.444|0.355|0.865|13.1|0.254|2020|[68.0,36.5,34.3,6...|  0.4966999201980734|
|  7|   8|      Damian Lillard| 29|POR|  0.0|   23.0|   1010|0.023| 66|37.5|30.0| 4.3| 8.0|1.1|0.3|0.463|0.401|0.888|11.

In [38]:
predictions.filter(predictions['year'] == 2012).orderBy(col("prediction").desc()).show()

+---+----+-----------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+--------------------+
|_c0|Rank|           Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS| TRB| AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|            features|          prediction|
+---+----+-----------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+--------------------+
|  0|   1|     LeBron James| 27|MIA| 85.0| 1074.0|   1210|0.888| 62|37.5|27.1| 7.9| 6.2|1.9|0.8|0.531|0.362|0.771|14.5|0.298|2012|[62.0,37.5,27.1,7...| 0.49678601975064995|
|  1|   2|     Kevin Durant| 23|OKC| 24.0|  889.0|   1210|0.735| 66|38.6|28.0| 8.0| 3.5|1.3|1.2|0.496|0.387| 0.86|12.2| 0.23|2012|[66.0,38.6,28.0,8...| 0.22484238070866122|
|  2|   3|       Chris Paul| 26|LAC|  6.0|  385.0|   1210|0.318| 60|36.4|19.8| 3.6| 9.1|2.5|0.1|0.478|0.371|0.861|12.7|0.278|2012|[60.0

In [40]:
predictions.filter(predictions['year'] == 2004).orderBy(col("prediction").desc()).show()

+---+----+----------------+---+---+-----+-------+-------+-----+---+----+----+----+---+---+---+-----+-----+-----+----+-----+----+--------------------+-------------------+
|_c0|Rank|          Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS| TRB|AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|            features|         prediction|
+---+----+----------------+---+---+-----+-------+-------+-----+---+----+----+----+---+---+---+-----+-----+-----+----+-----+----+--------------------+-------------------+
|  0|   1|   Kevin Garnett| 27|MIN|120.0| 1219.0|   1230|0.991| 82|39.4|24.2|13.9|5.0|1.5|2.2|0.499|0.256|0.791|18.3|0.272|2004|[82.0,39.4,24.2,1...| 0.5294308861050006|
|  1|   2|      Tim Duncan| 27|SAS|  0.0|  716.0|   1230|0.582| 69|36.6|22.3|12.4|3.1|0.9|2.7|0.501|0.167|0.599|13.1|0.249|2004|[69.0,36.6,22.3,1...|0.23461067748526976|
|  5|   6|Shaquille O'Neal| 31|LAL|  0.0|  178.0|   1230|0.145| 67|36.8|21.5|11.5|2.9|0.5|2.5|0.584|  0.0| 0.49| 9.9|0.192|2004|[67.0,36.8,21.5,1...| 

In [39]:
predictions.filter(predictions['year'] == 2008).orderBy(col("prediction").desc()).show()

+---+----+-----------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+--------------------+
|_c0|Rank|           Player|Age| Tm|First|Pts Won|Pts Max|Share|  G|  MP| PTS| TRB| AST|STL|BLK|  FG%|  3P%|  FT%|  WS|WS/48|year|            features|          prediction|
+---+----+-----------------+---+---+-----+-------+-------+-----+---+----+----+----+----+---+---+-----+-----+-----+----+-----+----+--------------------+--------------------+
|  3|   4|     LeBron James| 23|CLE|  1.0|  438.0|   1260|0.348| 75|40.4|30.0| 7.9| 7.2|1.8|1.1|0.484|0.315|0.712|15.2|0.242|2008|[75.0,40.4,30.0,7...|  0.5293313877562136|
|  1|   2|       Chris Paul| 22|NOH| 28.0|  894.0|   1260| 0.71| 80|37.6|21.1| 4.0|11.6|2.7|0.1|0.488|0.369|0.851|17.8|0.284|2008|[80.0,37.6,21.1,4...|  0.3544131326967456|
|  0|   1|      Kobe Bryant| 29|LAL| 82.0| 1100.0|   1260|0.873| 82|38.9|28.3| 6.3| 5.4|1.8|0.5|0.459|0.361| 0.84|13.8|0.208|2008|[82.0

In [41]:
from pyspark.sql.functions import min

last_10_mvp = mvp_voting.filter(mvp_voting['year'] > 2014)

# Calculate the minimum values for the specified columns across the entire DataFrame
min_values = last_10_mvp.agg(
    min("G").alias("Min_G"),
    min("MP").alias("Min_MP"),
    min("PTS").alias("Min_PTS"),
    min("TRB").alias("Min_TRB"),
    min("AST").alias("Min_AST"),
    min("STL").alias("Min_STL"),
    min("BLK").alias("Min_BLK"),
    min("FG%").alias("Min_FG%"),
    min("3P%").alias("Min_3P%"),
    min("FT%").alias("Min_FT%"),
    min("WS").alias("Min_WS"),
    min("WS/48").alias("Min_WS/48")
)

# Show the minimum values
min_values.show()


+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+---------+
|Min_G|Min_MP|Min_PTS|Min_TRB|Min_AST|Min_STL|Min_BLK|Min_FG%|Min_3P%|Min_FT%|Min_WS|Min_WS/48|
+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+---------+
|   44|  25.6|   13.9|    2.6|    1.3|    0.6|    0.1|  0.419|    0.0|  0.613|   3.1|    0.075|
+-----+------+-------+-------+-------+-------+-------+-------+-------+-------+------+---------+



In [32]:
from pyspark.sql.functions import col

#Clear out the players that are highly possible not in the conversation of MVP awards.

eligible_player = final_df.filter(
    (col('predict_G') >= 44) & 
    (col('predict_MP') >= 25.6) & 
    (col('predict_PTS') >= 13.9) & 
    (col('predict_TRB') >= 2.6) & 
    (col('predict_AST') >= 1.3) & 
    (col('predict_STL') >= 0.6) & 
    (col('predict_BLK') >= 0.1) & 
    (col('predict_FG%') >= 0.419) & 
    (col('predict_3P_Ptg') >= 0.0) & 
    (col('predict_FT%') >= 0.613) & 
    (col('predict_WS') >= 3.1) & 
    (col('predict_WS/48') >= 0.075)
)


In [33]:
column_mapping = {
    'predict_G': 'G',
    'predict_MP': 'MP',
    'predict_3P_Ptg': '3P%',
    'predict_TRB': 'TRB',
    'predict_FG%': 'FG%',
    'predict_FT%': 'FT%',
    'predict_AST': 'AST',
    'predict_STL': 'STL',
    'predict_BLK': 'BLK',
    'predict_TOV': 'TOV',
    'predict_PTS': 'PTS',
    'predict_WS': 'WS',
    'predict_WS/48': 'WS/48'
}

for old_name, new_name in column_mapping.items():
    eligible_player = eligible_player.withColumnRenamed(old_name, new_name)


feature_columns = ['G', 'MP', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'FG%', '3P%', 'FT%', 'WS', 'WS/48']
target_column = 'Share'

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
df_transformed = assembler.transform(eligible_player)

mvp_predictions = rf_model.transform(df_transformed)


In [35]:
mvp_predictions.orderBy(col("prediction").desc()).show()

+----------------+-------------+--------------------+----+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+--------------------+--------------------+
|Player Reference|       Season|              Player| Age|                 G|                MP|                3P%|               TRB|                FG%|               FT%|               AST|               STL|                BLK|               TOV|               PTS|                WS|              WS/48|            features|          prediction|
+----------------+-------------+--------------------+----+------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+------------------+-------------------+------------------+------------------+------------------+-------